In [1]:
import gym
import numpy as np

ENV = gym.make("CartPole-v1")

NUM_FEATURES = 4
LAMBDA       = 5
SIGMA        = 0.1
NUM_TRIALS   = 1

def sample_population(mu):
    return mu + SIGMA * np.random.randn(LAMBDA, NUM_FEATURES)

def sample_action(sample, observation):
    return 1 if np.dot(observation, sample) > 0.0 else 0

def evaluate(sample):
    total_rewards = np.zeros(NUM_TRIALS)
    for trial in range(NUM_TRIALS):
        total_reward = 0.0
        observation = ENV.reset()
        for step in range(500):
            action = sample_action(sample, observation)
            observation, reward, done, _ = ENV.step(action)
            total_reward += reward
            if done: break
        total_rewards[trial] = total_reward
    return np.mean(total_rewards)

def update_mu(population):
    return population[0] # TODO, but this works fine for cartpole

def play(mu):
    observation = ENV.reset()
    for step in range(500):
        ENV.render()
        action = sample_action(mu, observation)
        observation, _, done, _ = ENV.step(action)
        if done:
            print("Episode finished after {} timesteps".format(step + 1))
            break

In [2]:
mu = np.zeros(NUM_FEATURES)

for episode in range(50):
    population = sample_population(mu)
    rewards = map(evaluate, population)
    rewards = np.fromiter(rewards, float)
    sort_idx = np.argsort(rewards)
    rewards = rewards[sort_idx][::-1]
    population = population[sort_idx][::-1]
    mu = update_mu(population)
print("rewards: {}".format(rewards))
    

rewards: [500. 500. 500. 422. 388.]


In [3]:
play(mu)

Episode finished after 500 timesteps
